In [1]:
import pandas as pd
import re 
import string

import io 
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

import pdfplumber

import requests

/home/noahdarwich/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
import tempfile 
from oauth2client.service_account import ServiceAccountCredentials

from googleapiclient.http import MediaIoBaseDownload

In [3]:
from arabic_reshaper import ArabicReshaper
from bidi.algorithm import get_display

In [4]:
from camel_tools.utils.dediac import dediac_ar
from ar_corrector.corrector import Corrector

In [5]:
from pdfquery import PDFQuery
from io import BytesIO, StringIO
from PIL import Image

import pytesseract

from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
from PyPDF2 import PdfFileReader, PdfFileWriter
import pyarabic.araby as araby

import camelot

from unidecode import unidecode

**Extract text from pdf**

In [6]:
def extract_text_from_pdf(pdf_content):
  resource_manager = PDFResourceManager()
  fake_file_handle = io.StringIO()
  converter = TextConverter(resource_manager, fake_file_handle)
  page_interpreter = PDFPageInterpreter(resource_manager, converter)

  with io.BytesIO(pdf_content) as fh:
    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
      page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

  # close open handles
  converter.close()
  fake_file_handle.close()

  return text

In [7]:
df_results = pd.read_csv('df_results.csv', index_col=0)
df_results

,issue,url,date,pdf_link,status
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...,NaN
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,NaN,not_saved
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...,NaN
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...,NaN
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...,NaN
...,...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...,saved
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...,saved
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...,saved
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...,saved


**Create folder in Google Drive**

In [62]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)
folder_id = '1BVzI84DDDtbvU3pp1k0yERF5YKIs84R0'
results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()

files = results.get("files", [])
next_page_token = results.get("nextPageToken")

while next_page_token:
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
    files.extend(results.get("files", []))
    next_page_token = results.get("nextPageToken")
    
# Create new folder "keywords" inside "AAA"
folder_metadata = {'name': 'events', 'parents': [folder_id], 'mimeType': 'application/vnd.google-apps.folder'}
folder = service.files().create(body=folder_metadata).execute()
events_folder_id = folder.get('id')

**Read files in Google drive (doesn't work for pdfs)**

In [66]:
# results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
# files = results.get('files', [])

# # Iterate through the files
# for file in files:
#     file_id = file.get('id')
#     file_name = file.get('name')
#     request = service.files().export_media(fileId=file_id,mimeType='application/pdf')
#     fh = io.BytesIO()
#     downloader = MediaIoBaseDownload(fh, request)
#     done = False
    #while done is False:
        #status, done = downloader.next_chunk()
        #print("Download %d%%." % int(status.progress() * 100))
    #fh.seek(0)
    # with open("downloaded_file.pdf", 'wb') as f:
    #     content = fh.read()
    #     print(content)

**Move file in Google drive**

In [ ]:
# # search for specific keywords in the content
# if 'keyword1' in content or 'keyword2' in content:
#     # move the file to the "keywords" folder
#     file_metadata = {'parents': [events_folder_id]}
#     service.files().update(fileId=file_id, body=file_metadata).execute()


**Correct Arabic Text**

In [56]:
# reshaper = ArabicReshaper(configuration=configuration)
# corrected_text = reshaper.reshape(text)
# corrected_text = get_display(text)
# corrected_text = corrected_text.replace('ـ', '')
# corrected_text

In [29]:
# from chardet.universaldetector import UniversalDetector
# detector = UniversalDetector()

In [30]:
test_sample = df_results[0:5]
test_folder_id = '19Nx9P1ekaDyoX06z2sj0FKokfcskouNc'

In [31]:
configuration = {
    'delete_harakat': True,
    'delete_tatweel ': True,
}
url = test_sample.iloc[0]['pdf_link']
url

'https://www.al-akhbar.com/PDF_Files/3922/alakhbar20191202.pdf'

In [75]:
response = requests.get(url)

with open('file.pdf', 'wb') as f:
    f.write(response.content)
    
pdf = PdfFileReader(open('file.pdf', 'rb'))
texts = {}

for page in range(0,pdf._get_num_pages()):
    with open('file.pdf', 'rb') as f:
        laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
        outfp = StringIO()
        outfp.seek(0)
        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
        outfp.seek(0)
        text = outfp.read()
        reshaper = ArabicReshaper(configuration=configuration)
        corrected_text = get_display(text)
        reshaped_text = reshaper.reshape(corrected_text)
        #reshaped_text = dediac_ar(reshaped_text).strip()
        #texts.append(reshaped_text)
        # print('-'*50, page)
        # print(reshaped_text)
        page_number = ''.join(x for x in reshaped_text[:4] if x in string.digits)
        #corr = Corrector().
        #corrected = corr.contextual_correct(reshaped_text)
        texts[page_number] = reshaped_text.strip().replace('\n',' ').replace('ـ', '')

        # keywords = ['عتصم', 'حتج']
        # for keyword in keywords:
            
        #regex = re.compile('|'.join(keywords), re.IGNORECASE)
        #match = regex.search(reshaped_text)
        #if any(keyword in reshaped_text for keyword in keywords):
            #if keyword in keywords:
            #if regex.search(reshaped_text):
                #new_file_name = match.group() + '.pdf'
                # output_pdf = PdfFileWriter()
                # output_pdf.addPage(pdf.getPage(page))
                # with open('blabla.pdf', 'wb') as f:
                #     output_pdf.write(f)
                    #print(page_number)
            # else:
            #     print('not_found')
                # print(page_number)

texts

{'32': '32 ﺻﻔﺤﺔ 1000 ﻟﻴﺮﺓ  ﺍﺍﻟﺜﻨﻴﻦ  2  ﻛﺎﻧﻮﻥ ﺍﺃﻟﻮﻝ  2019 ﺍﻟﻌﺪﺩ  3922  ﺍﻟﺴﻨﺔ  ﺍﻟﺮﺍﺑﻌﺔ ﻋﺸﺮﺓ lundi  2  Décembre  2019  no 3922   14ème année  ﺭﺃﺱ ﺍﻟﻤﺎﻝ ﺍﺣﺘﻴﺎﻁ »ﺍﻟﻤﺮﻛﺰﻱ«  ﺑﺎﻟﺪﻭﺍﻟﺮ ﻳﺘﺎﻟﺸﻰ ● ﻣﺤﻤﺪ ﻭﻫﺒﺔ ﻧﻤﻮﺫﺝ ﺍﻗﺘﺼﺎﺩﻱ ﺹ ﻡ ﻡ ﻟﻘﺘﻞ ﺍﻟﺰﺭﺍﻋﺔ  ● ﻋﺒﺪ ﺍﻟﺤﻠﻴﻢ ﻓﻀﻞ ﷲ ﻋﻦ ﺧﺮﺍﻓﺔ ﺍﻟﻨﻤﻮﺫﺝ ﻭﺣﻜﺎﻳﺔ ﺍﻧﻬﻴﺎﺭ ﻣﺆﺝ ﻝ  ﺃﺳﺒﻮﻉ ﺣﺎﺳﻢ ﺣﻜﻮﻣﻴﺎ  ]2[  ﺍﻟﻔﻮﺿﻰ  ﺗﻀﺮﺏ ﺍﻟﻌﺮﺍﻕ  ]14   15[  ) ﺃ  ﻑ ﺏ  (  ﺗﺮﻛﻴﺎ ﻭﻟﻴﺒﻴﺎ ﻭﻏﺎﺯ  ﺍﻟﻤﺘﻮﺱ ﻁ »ﺍﺁﻟﻦ ﺑﺪﺃﺕ ﺍﻟﻠﻌﺒﺔ«!  18  ﻗﻀﻴﺔ  ﺍﻟﻴﻤﻦ  ﻣﻘﺎﺑﻠﺔ   ﺳﻌﻮﺩﻱ   ﺿﻐﻂ ﺇﻟﻨﺸﺎﺀ ﻣﻨﻄﻘﺔ  ﻋﺎﺯﻟﺔ 16  ﺇﻳﻠﻲ ﻣﺸﻨﺘﻒ ﻧﺘﻤﻨﻰ ﺃﻥ ﻳﺒﻘﻰ  ﺍﻟﺒﻠﺪ ﻟﺘﺒﻘﻰ ﺍﻟﻠﻌﺒﺔ  8',
 '3': '3  ﺍﺇﻟﺜﻨﻴﻦ  2  ﻛﺎﻧﻮﻥ ﺍﺃﻟﻮﻝ  2019  ﺍﻟﻌﺪﺩ  82  ﺍﻟﻌﺒﻮﺭ ﻧﺤﻮ ﻧﻤﻮﺫﺝ ﺗﻨﻤﻮﻱ  ﺑﺪﻳﻞ  ﺑﺎﻟﺨﻔﺾ، ﺫﻟﻚ ﺃﻥ  ﺃﻱ  ﺍﺭﺗﻔﺎﻉ ﻓﻲ ﺃﺳﻌﺎﺭ  ﺍﻟﺴﻠﻊ ﺍﻟﻮﻃﻨﻲ ﺓ ﺍﻟﻘﺎﺑﻠﺔ ﻟﻠﺘﺒﺎﺩﻝ ﻳﺴﺎﻫﻢ ﻓﻲ  ﺍﻣﺘﺼﺎﺹ ﺍﻟﺨﻔﺾ ﺍﻟﺤﺎﺻﻞ ﻟﻘﻴﻤﺔ ﺍﻟﻌﻤﻠﺔ  ﺍﻟﻮﻃﻨﻲ ﺔ، ﻭﻓﻲ ﺇﻟﻐﺎﺀ ﻣﻔﺎﻋﻴﻞ ﺍﻧﺨﻔﺎﺽ  ﺃﺳﻌﺎﺭ ﺍﻟﺼﺎﺩﺭﺍﺕ ﺍﻟﻮﻃﻨﻲ ﺓ ﻣﻘﻮ ﻣﺔ ﺑﺎﻟﻌﻤﻠﺔ  ﺍﺃﻟﺠﻨﺒﻴﺔ.   ﻭﺃﺧﻴﺮﺍ، ﻳﻨﺒﻐﻲ ﺃﻥ ﺗﺖ ﺴﻢ ﺍﻣﻠﻨﺘﺠﺎﺕ  ﺍﻟﻮﻃﻨﻲ ﺓ  ﺍﻟﻘﺎﺑﻠﺔ  ﻟﻠﺘﺒﺎﺩﻝ  ﺑﺎﻟﺠﻮﺩﺓ  ﻭﺍﻟﺘﻨﻮ ﻉ،  ﻛﻤﺎ ﺍﻟﻘﺪﺭﺓ ﻋﻠﻰ ﺍﻣﻠﻨﺎﻓﺴﺔ ﻏﻴﺮ ﺍﻟﺴﻌﺮﻱ ﺓ،  ﺍﺃﻟﻤﺮ ﺍﻟﺬﻱ ﻣﻦ ﺷﺄﻧﻪ ﺃﻥ ﻳﺴﻬﻢ ﻓﻲ ﺯﻳﺎﺩﺓ  ﺏ ﺫﻟﻚ ﺃﻳﻀﺎ  ﺍﻟﻄﻠﺐ ﺍﻟﺨﺎﺭﺟﻲ ﻋﻠﻴﻬﺎ. ﻭﻳﺘﻄﻞ ﻭﺟﻮﺩ ﻣﻬ

In [88]:
keywords = ['عتصم', 'حتج', 'رأس المال']
#regex = re.compile('|'.join(keywords), re.IGNORECASE)
#print(texts)
pdf = PdfFileReader(open('file.pdf', 'rb'))
for page in range(0,pdf._get_num_pages()):
    for key, value in texts.items():
        #match = regex.search(value)
        #print(match)
        words = value.split()
        #print(words)
        for keyword in keywords:
            for word in words:
                if keyword in word:
                    print(value)
                    # output_pdf = PdfFileWriter()
                    # output_pdf.addPage(pdf.getPage(page))
                    # with open('single_odf.pdf', 'wb') as f:
                    #     output_pdf.write(f)

In [13]:
corr = Corrector()
corr.spell_correct('ﺇﻟﻨﺸﺎء ',2)

'ﺇﻟﻨﺸﺎﺀ '

In [63]:
import codecs

In [67]:
pdf_string = "ﺍﻟﺴﻨﺔﺍﻟﺮﺍﺑﻌﺔﻋﺸﺮﺓ"
ex= "السنةالرابعةعشرة"
pdf_string_encoded = codecs.decode(codecs.encode(pdf_string))
ex_encoded = codecs.decode(codecs.encode(ex))
#pdf_string_decoded = pdf_string_encoded.decode("cp1251")

print(pdf_string_encoded)
print(ex_encoded)

if pdf_string_encoded == ex_encoded:
    print("Strings match!")
else:
    print("Strings don't match.")
# def clean_text(text):
#     text = text.strip()
#     text = text.replace('\n', '')
#     text = text.replace('\r', '')
#     text = text.replace('\t', '')
#     return text
# cleaned_text = clean_text(pdf_string)
# cleaned_text = pdf_string.encode('utf-8')
# expected_string_encoded = expected_string.encode('utf-8')

ﺍﻟﺴﻨﺔﺍﻟﺮﺍﺑﻌﺔﻋﺸﺮﺓ
السنةالرابعةعشرة
Strings don't match.


In [28]:
codecs. .getdecoder('utf-8')

<function encodings.utf_8.decode(input, errors='strict')>

In [23]:
encodings = ["utf-16-le", "utf-16-be", "utf-8", "ascii"]

for encoding in encodings:
    try:
        decoded_string = codecs.decode(pdf_string, encoding)
        print("The original encoding is likely:", encoding)
        print("Decoded text:", decoded_string)
        break
    except:
        print("Failed to decode using", encoding)

The original encoding is likely: utf-16-le
Decoded text: 뫯龻뫯ꢻ뫯趺믯꺺뫯醺믯钺믯뢺뫯鎺


In [89]:
from charset_normalizer import from_path

results = from_path('./file.pdf')

print(str(results.best()))

None


In [98]:
string1 = pdf_string.encode()
string2 = ex.encode()

if string1.decode('Windows-1252') == string2.decode('Windows-1252'):
    unified_string = string1.decode('Windows-1252').encode('utf-8')
    print("Strings are the same")
else:
    print("Strings are different")

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 2: character maps to <undefined>

In [59]:
'ﺧﺮﺍﻓﺔ' == 'خرافة'

False

In [35]:
keywords = ['ﺗﺮﻛ ', 'asdasd', 'ghghgh']

In [36]:
for x in texts['32'].split():
    for key in keywords:
        if key in x.strip():
            print(x)

In [26]:
keywords = ['عتصم', 'حتج', 'ﺗﻨﻤﻮﻱ']
regex = re.compile('|'.join(keywords), re.IGNORECASE)

for keyword in keywords:
    
    for key, value in texts.items():
        #match = regex.search(value)
        if keyword in value:
            output_pdf = PdfFileWriter()
            output_pdf.addPage(pdf.getPage(int(key)))
            with open('single_odf.pdf', 'wb') as f:
                output_pdf.write(f)
                print('yes')
        # else:
        #     print('Not_found')

yes


In [17]:
response = requests.get(url)

with open('file.pdf', 'wb') as f:
    f.write(response.content)

with open('file.pdf', 'rb') as f:
    laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
    outfp = StringIO()
    outfp.seek(0)
    for page in range(0, 10):
        outfp.seek(0)
        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
        outfp.seek(0)
        text = outfp.read()
        reshaper = ArabicReshaper(configuration=configuration)
        corrected_text = get_display(text)
        reshaped_text = reshaper.reshape(corrected_text)
        # print('-'*50, page)
        #print(reshaped_text)
        
        if 'ﺃﻟﻐﺎﺛﺎ ﻛﺮﻳﺴﺘﻲ'in reshaped_text:
            pdf = PdfFileReader(f)
            output_pdf = PdfFileWriter()
            output_pdf.addPage(pdf.getPage(page))
            with open('single_page.pdf', 'wb') as f:
                output_pdf.write(f)
        else:
            print('not_found')

32 ﺻﻔﺤﺔ
1000 ﻟﻴﺮﺓ

ﺍﺍﻟﺜﻨﻴﻦ  2  ﻛﺎﻧﻮﻥ ﺍﺃﻟﻮﻝ  2019
ﺍﻟﻌﺪﺩ  3922  ﺍﻟﺴﻨﺔ  ﺍﻟﺮﺍﺑﻌﺔ ﻋﺸﺮﺓ
lundi  2  Décembre  2019  no 3922   14ème année

ﺭﺃﺱ ﺍﻟﻤﺎﻝ
ﺍﺣﺘﻴﺎﻁ »ﺍﻟﻤﺮﻛﺰﻱ« 
ﺑﺎﻟﺪﻭﺍﻟﺮ ﻳﺘﺎﻟﺸﻰ
● ﻣﺤﻤﺪ ﻭﻫﺒﺔ
ﻧﻤﻮﺫﺝ ﺍﻗﺘﺼﺎﺩﻱ
ﺹ ﻡ ﻡ ﻟﻘﺘﻞ ﺍﻟﺰﺭﺍﻋﺔ

● ﻋﺒﺪ ﺍﻟﺤﻠﻴﻢ ﻓﻀﻞ ﷲ
ﻋﻦ ﺧﺮﺍﻓﺔ ﺍﻟﻨﻤﻮﺫﺝ
ﻭﺣﻜﺎﻳﺔ ﺍﻧﻬﻴﺎﺭ ﻣﺆﺝ ﻝ

ﺃﺳﺒﻮﻉ ﺣﺎﺳﻢ ﺣﻜﻮﻣﻴﺎ  ]2[

ﺍﻟﻔﻮﺿﻰ 
ﺗﻀﺮﺏ ﺍﻟﻌﺮﺍﻕ

]14 ـ  15[

)
ﺃ

ﻑ
ﺏ

(

ﺗﺮﻛﻴﺎ ﻭﻟﻴﺒﻴﺎ ﻭﻏﺎﺯ 
ﺍﻟﻤﺘﻮﺱ ﻁ
»ﺍﺁﻟﻦ ﺑﺪﺃﺕ ﺍﻟﻠﻌﺒﺔ«!

18

ﻗﻀﻴﺔ

ﺍﻟﻴﻤﻦ

ﻣﻘﺎﺑﻠﺔ

 ﺳﻌﻮﺩﻱ  
ﺿﻐﻂ
ﺇﻟﻨﺸﺎﺀ ﻣﻨﻄﻘﺔ 
ﻋﺎﺯﻟﺔ
16

ﺇﻳﻠﻲ ﻣﺸﻨﺘﻒ
ﻧﺘﻤﻨﻰ ﺃﻥ ﻳﺒﻘﻰ 
ﺍﻟﺒﻠﺪ ﻟﺘﺒﻘﻰ ﺍﻟﻠﻌﺒﺔ

8


 
 

not_found
3

ﻟﺒﻨﺎﻥ

ﺍﺇﻟﺜﻨﻴﻦ  2  ﻛﺎﻧﻮﻥ ﺍﺃﻟﻮﻝ  2019  ﺍﻟﻌﺪﺩ  3922

ﻋﺖ 

ﺭ ﻭﺧﻂ

ﻗﺔ ﺑﺄﺳﻤﺎﺀ ﻫﺬﻩ ﺍﻟﺠﻤﻌﻴﺎﺕ 

ﻡ ﻭﻓﺮﺹ ﺍﻟﻌﻤﻞ ﻭﺍﻟﺪﻭﺭ ﺍﻟﻘﻴﺎﺩﻱ، ﻭﺭﻋﺎﻳﺔ 

ﻭﻣﺎ ﺍﻟﺬﻱ ﺗﻤﻠﻜﻪ ﺍﺁﻟﻦ، ﻭﺻﻮﺭﺓ ﻣﺒﺲ ﻃﺔ ﻋﻦ ﻧﻈﺎﻡ ﺣﻴﺎﺗﻬﺎ ﺍﻟﻌﺎﻣﺔ، 
ﻣﺜﻞ ﺃﻳﻦ ﺗﻌﻴﺶ ﻭﺃﻳﻦ ﻳﺘﻌﻠﻢ ﺍﺃﻟﺒﻨﺎﺀ ﻭﺃﻳﻦ ﻳﻘﻀﻮﻥ ﺍﺇﻟﺠﺎﺯﺍﺕ ﻭﻧﻮﻉ 
ﺍﻟﺴﻴﺎﺭﺍﺕ ﻭﺍﻟﺨﺪﻣﺎﺕ ﺍﻟﺘﻲ ﻳﺤﺼﻠﻮﻥ ﻋﻠﻴﻬﺎ ﻓﻲ ﺣﻴﺎﺗﻬﻢ ﺍﻟﻴﻮﻣﻴﺔ؟
، ﻣﻦ ﺃﻳﻦ ﻱ ﻧﻔﻖ ﺣﺰﺏ ﺍﻟﻜﺘﺎﺋﺐ ﻃﻮﺍﻝ ﺍﻟﻌﻘﻮﺩ ﺍﻣﻼﺿﻴﺔ، ﻭﻣﺎ ﻫﻲ 
ﻣﺜﺎﻝ
ﺍﻣﻠﻬﻦ ﺍﻟﺘﻲ ﻳﻘﻮﻡ ﺑﻬﺎ ﺁﻝ ﺍﻟﺠﻤﻴﻞ ﺑﻜﻞ ﻓﺮﻭﻋﻬﻢ؟ ﺃﻭ ﻟﻨﺴﺄﻝ ﺣﺰﺏ 
ﻓﻘﺮﺍﺀ ﺍﻣﻠﺴﻴﺤﻴﻨﻲ، ﺃﻱ ﺍﻟﻘﻮﺍﺕ ﺍﻟﻠﺒﻨﺎﻧﻴﺔ، ﻋﻦ ﻣﺼﺪﺭ ﻧﻔﻘﺎﺗﻪ ﺍﻟﺸﻬﺮﻳ

In [60]:

# pdf_response = requests.get(url)

# with open('file.pdf', 'wb') as f:
#     f.write(pdf_response.content)

# with open('file.pdf', 'rb') as f:
    # pdf = PDFQuery(f)
    # pdf.load()
    # blocks = pdf.extract([
    #     ('with_parent','LTPage'),
    #     ('block_txt','LTTextLineHorizontal')
    # ])
    # for block in blocks:
    #     print(block.text.strip())
    # pdf = PdfFileReader(f)

    # outfp = StringIO()
    # laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
    
    # for page in range(1,pdf.getNumPages()):
    #     outfp.seek(0)
    #     extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
    #     outfp.seek(0)
    #     text = outfp.read()
    #     reshaper = ArabicReshaper(configuration=configuration)
    #     #corrected_text = reshaper.reshape(text)
    #     corrected_text = get_display(text)
    #     print(corrected_text)
    #     if 'أسبوع' in corrected_text:
    #         print('TADAAAAAAAAAAAAAAA',page)
    #         break


3

لبنان

اإلثنين  2  كانون األول  2019  العدد  3922

عت 

ر وخط

قة بأسماء هذه الجمعيات 

م وفرص العمل والدور القيادي، ورعاية 

وما الذي تملكه اآلن، وصورة مبسّ طة عن نظام حياتها العامة، 
مثل أين تعيش وأين يتعلم األبناء وأين يقضون اإلجازات ونوع 
السيارات والخدمات التي يحصلون عليها في حياتهم اليومية؟
، من أين يُ نفق حزب الكتائب طوال العقود املاضية، وما هي 
مثال
املهن التي يقوم بها آل الجميل بكل فروعهم؟ أو لنسأل حزب 
فقراء املسيحيني، أي القوات اللبنانية، عن مصدر نفقاته الشهرية 
والسنوية في كل القطاعات، بما في ذلك كلفة حياة قائده 
وخدماته وحراسته. واألمر نفسه بالنسبة إلى التيار الوطني 
الحر وحركة أمل، والزعيم الوطني الكبير املفدّ ى وليد جنبالط، 
وحجم اإلنفاق القائم على الدولة لحراستهم وتأمني خدماتهم 
اللوجستية؟. وماذا عن بقية الزعامات، من بقايا اإلقطاع 
األسعدي في الجنوب، إلى مراعبة عكار، مرورًا بآل فرنجية 
ومعوض والخازن وفرعون والصحناوي، حتى آل الحريري في 
هذه األيام...؟
ماذا عن كل جمعيات دعم املرأة وتمكينها وحريتها ورفض 
تعنيفها، ومقاومة التمييز الجندري، وحق األمومة والجنسية، 
واملساواة

In [58]:
# text_ls = [] 
# for link in pdf_links:
#     # download the PDF
#     pdf_url = link['href']
    
#     pdf_content = pdf_response.content
#     # extract the text from the PDF
#     text = extract_text_from_pdf(pdf_content)
#     text_ls.append(text)
    
# print(soup)

In [59]:
# url = df.iloc[0]['pdf_link']
# pdf_response = requests.get(url)
# pdf_content = pdf_response.content
# text = extract_text_from_pdf(pdf_content)
# text